<h1 align=center><font size = 5>Toronto Clustering Assignment by Ryan Diaz-Perez</font></h1>

<h1 align=left><font size = 4>Part 1: Web Scraping</font></h1>

In [1]:
!conda install lxml --yes
!conda install html5lib --yes
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.3.0                 |   py36hefd8a0e_0         1.5 MB
    certifi-2019.11.28         |           py36_0         156 KB
    openssl-1.1.1d             |       h7b6447c_3         3.7 MB
    ------------------------------------------------------------
                                           Total:         5.4 MB

The following packages will be UPDATED:

    certifi: 2019.9.11-py36_0     conda-forge --> 2019.11.28-py36_0   
    lxml:    4.2.5-py37hefd8a0e_0             --> 4.3.0-py36hefd8a0e_0
    openssl: 1.1.1d

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from sklearn.cluster import KMeans # import k-means from clustering stage
import html5lib
from bs4 import BeautifulSoup
import lxml

<Big><b>Extracting data from the Wikipedia page</b></Big>

In [3]:
url = "http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df_toronto = pd.read_html(url, flavor='html5lib')[0]

# Removing rows with Borough value of Not assigned
df_toronto.drop(df_toronto.index[df_toronto['Borough'] == 'Not assigned'], inplace = True)
df_toronto.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [4]:
# Combining rows
df_toronto = df_toronto.groupby(by=['Postcode','Borough']).agg(lambda x: ','.join(x))
df_toronto.reset_index(level=['Postcode','Borough'], inplace=True)
df_toronto.head(210)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


In [5]:
df_toronto.loc[df_toronto['Neighbourhood'] == ('Not assigned'), 'Neighbourhood'] = df_toronto['Borough']
df_toronto.head(90)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
85,M7A,Queen's Park,Queen's Park
86,M7R,Mississauga,Canada Post Gateway Processing Centre
87,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
88,M8V,Etobicoke,"Humber Bay Shores,Mimico South,New Toronto"


In [6]:
df_toronto.shape

(103, 3)

<h1 align=left><font size = 4>Part 2: Geographical locations</font></h1>

In [25]:
# Latitude and Longitude of the neighborhoods
df_geography = pd.read_csv('Geospatial_Coordinates.csv')
df_geography.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
# Renaming colunm from Postal Code to Postcode
df_geography.rename({'Postal Code': 'Postcode'}, axis=1, inplace=True)
df_geography.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [27]:
# Merging df_toronto and df_geography
df_merged = df_toronto.merge(df_geography, how='left', on='Postcode')
df_merged.rename({'Postcode': 'PostalCode'}, axis=1, inplace=True)
df_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<h1 align=left><font size = 4>Part 3: Maps of the neighborhood</font></h1>